In [17]:
import pandas as pd
import numpy as np
import pickle
import os
import copy


### VISUALIZANDO O DATASET FORNECIDO

In [18]:
df = pd.read_csv('./Ansatz_supervised_training_reports(CostaSH).csv')

In [19]:
df.head()

,ARQUITETURA_ANSATZ,INPUT_EMBEDDING,DATASET,DATASET_DIVISION_INDEX,OPTIMIZER,UNSUPERVISED_METRIC,MEASURED_WIRES,MEASURE_TYPE,TRAIN_METRIC_COST,TEST_METRIC_COST,TRAIN_ACCURACY,TEST_ACCURACY,WEIGHT,BIAS,USE_BIAS
0,ry,amplitude,xor_50samples_no_noise,0,SPSAOptimizer(10),supervised,1,expval,0.302347,0.377357,0.075087,-0.025316,[0.53625616],[0.10402457],YES
1,ry,amplitude,xor_50samples_no_noise,0,GradientDescentOptimizer(1),supervised,1,expval,0.143496,0.132293,0.625850,0.637273,[1.67174535],[0.64652642],YES
2,ry,amplitude,xor_50samples_no_noise,0,NesterovMomentumOptimizer(0.1),supervised,1,expval,0.143032,0.135463,0.693814,0.809091,[1.60658165],[0.60539028],YES
3,ry,amplitude,xor_50samples_no_noise,1,SPSAOptimizer(10),supervised,1,expval,0.288678,0.322704,0.103410,0.069620,[0.52921011],[0.12613722],YES
4,ry,amplitude,xor_50samples_no_noise,1,GradientDescentOptimizer(1),supervised,1,expval,0.130292,0.171823,0.765306,0.350909,[1.57184023],[0.67975578],YES


In [20]:
# Valores únicos de todas as colunas
valores_unicos = {col: df[col].unique() for col in df.columns}

# Exibindo os resultados
for col, valores in valores_unicos.items():
    print(f"\nValores únicos em {col}:")
    print(valores)
    print(f"Número de valores únicos: {len(valores)}")


Valores únicos em ARQUITETURA_ANSATZ:
['ry' 'rx' 'rz' 'ru' 'hr' 'ruu' '4_wana' '8_nake' '9_divwe' '1_mochi'
 '5_tano' '7_sambwadi' 'E' 'B' 'H' 'phi' 'beta' 'G' 'A' 'C' 'D' 'M' 'K'
 'F' 'J' 'I' 'N' 'O' 'L' 'gamma' 'theta' 'alpha' '0_zero' '3_tatu'
 '2_yadi' '6_samanu']
Número de valores únicos: 36

Valores únicos em INPUT_EMBEDDING:
['amplitude' 'phasex' 'phasey']
Número de valores únicos: 3

Valores únicos em DATASET:
['xor_50samples_no_noise' 'xor_500samples_low_noise'
 'xor_500samples_no_noise' 'xor_50samples_low_noise'
 'circles_500samples_low_noise' 'circles_500samples_no_noise'
 'circles_50samples_low_noise' 'circles_50samples_no_noise'
 'moons_500samples_low_noise' 'moons_500samples_no_noise'
 'moons_50samples_no_noise' 'moons_50samples_low_noise'
 'blobs_2classes_2features_50samples_low_noise'
 'blobs_2classes_2features_500samples_low_noise'
 'blobs_2classes_2features_50samples_no_noise'
 'blobs_2classes_2features_500samples_no_noise'
 'blobs_3classes_2features_50samples_low_no

In [21]:
# ordenando o DataFrame por 'ARQUITETURA_ANSATZ'
df_sorted = df.sort_values(by=['ARQUITETURA_ANSATZ'])

In [22]:
# verificando a ordem dos valores únicos de 'ARQUITETURA_ANSATZ' após a ordenação
valores_unicos_sorted = df_sorted['ARQUITETURA_ANSATZ'].unique()
print("\nValores únicos de 'ARQUITETURA_ANSATZ' após ordenação:")
print(valores_unicos_sorted)
print(f"Número de valores únicos após ordenação: {len(valores_unicos_sorted)}")


Valores únicos de 'ARQUITETURA_ANSATZ' após ordenação:
['0_zero' '1_mochi' '2_yadi' '3_tatu' '4_wana' '5_tano' '6_samanu'
 '7_sambwadi' '8_nake' '9_divwe' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J'
 'K' 'L' 'M' 'N' 'O' 'alpha' 'beta' 'gamma' 'hr' 'phi' 'ru' 'ruu' 'rx'
 'ry' 'rz' 'theta']
Número de valores únicos após ordenação: 36


### TRANSFORMANDO O DATASET PARA O INPUT DOS MODELOS

In [23]:
def transformar_dataframe(df, file_path, file_metrics = "./../datasets/metrics/",  target="ACURACIA_TESTE"):

    """
    Função para transformar os metadados do treinamento do ansatz nos dados para os modelos clássicos.
    Parâmetros: df: DataFrame com os metadados do treinamento do ansatz
              file_metrics: caminho do arquivo com os metadados do treinamento do ansatz
              file_path: caminho para salvar o arquivo csv
              target: nome da coluna alvo (default: "ACURACIA_TESTE")
    Retorna: Arquivos csv com os dados transformados
    """
    # ordenando o DataFrame por 'ARQUITETURA_ANSATZ'
    df = df.sort_values(by=['ARQUITETURA_ANSATZ'])

    # mapeamento dos valores únicos de 'ARQUITETURA_ANSATZ' para números
    ansatz_mapping = {val: i for i, val in enumerate(df['ARQUITETURA_ANSATZ'].unique())}

    # mapeamento do valores únicos de 'INPUT_EMBEDDING' para números
    embedding_mapping = {val: i for i, val in enumerate(df['INPUT_EMBEDDING'].unique())}

    # mapeamento do valores únicos de 'T' para números
    unsupervised_mapping = {val: i for i, val in enumerate(df['UNSUPERVISED_METRIC'].unique())}

    #mapeamento do valores únicos de 'OPTIMIZER' para números
    optimizer_mapping = {val: i for i, val in enumerate(df['OPTIMIZER'].unique())}

    # exibindo os mapeamentos
    print("\nMapeamento de 'ARQUITETURA_ANSATZ' para números:")
    for val, i in ansatz_mapping.items():
        print(f"{val}: {i}")
    print("\nMapeamento de 'INPUT_EMBEDDING' para números:")
    for val, i in embedding_mapping.items():
        print(f"{val}: {i}")
    print("\nMapeamento de 'UNSUPERVISED_METRIC' para números:")
    for val, i in unsupervised_mapping.items():
        print(f"{val}: {i}")
    print("\nMapeamento de 'OPTIMIZER' para números:")
    for val, i in optimizer_mapping.items():
        print(f"{val}: {i}")


    # nomes dos datasets
    datasets = df['DATASET'].unique()

    # Criando uma lista para armazenar as métricas  
    metrics = []

    for dataset in datasets:
        # Carregando as métricas
        with open(file_metrics + dataset + ".pkl", 'rb') as file:
            aux_metrics = pickle.load(file)
        # Adicionando as métricas à lista
        metrics.append(aux_metrics)

    new_df = {
        'f1': [],
        'f1v': [], 
        'f2': [], 
        'f3': [], 
        'f4': [], 
        'l1': [], 
        'l2': [], 
        'l3': [], 
        'n1': [], 
        'n2': [], 
        'n3': [], 
        'n4': [], 
        't1': [], 
        'lsc': [], 
        'density': [], 
        'clsCoef': [], 
        'hubs': [], 
        't2': [], 
        't3': [], 
        't4': [], 
        'c1': [], 
        'c2': [],
        'target': []
    }


    for sample_id in range(5):

        #filtrar pelo número da sample
        df_sample = df[df["DATASET_DIVISION_INDEX"]==sample_id]

        df_best_accuracy = copy.deepcopy(new_df)
        df_best_ansatz = copy.deepcopy(new_df)
        df_unsupervised_metric = copy.deepcopy(new_df)
        df_best_embedding = copy.deepcopy(new_df)
        df_best_optimizer = copy.deepcopy(new_df)

        for metric in metrics:

            #filtrar pelo nome do arquivo
            df_arq_sample = df_sample[df_sample["DATASET"]==metric["arquivo"]]

            #filtrar pelo Bias
            df_arq_sample_bias = df_arq_sample[df_arq_sample["USE_BIAS"]=="YES"]

            #preencher os dataframes
            for key in new_df.keys():

                if key != 'target':

                    df_best_accuracy[key].append(metric[key][sample_id])
                    df_best_ansatz[key].append(metric[key][sample_id])
                    df_unsupervised_metric[key].append(metric[key][sample_id])
                    df_best_embedding[key].append(metric[key][sample_id])
                    df_best_optimizer[key].append(metric[key][sample_id])

                else:
                    #preencher a coluna target

                    idx = df_arq_sample_bias[target].idxmax()

                    # para a melhor acurácia
                    df_best_accuracy[key].append(df_arq_sample_bias.loc[idx, target])

                    # para o melhor ansatz
                    df_best_ansatz[key].append(ansatz_mapping[df_arq_sample_bias.loc[idx, 'ARQUITETURA_ANSATZ']])

                    # para a melhor métrica não supervisionada
                    df_unsupervised_metric[key].append(unsupervised_mapping[df_arq_sample_bias.loc[idx, 'UNSUPERVISED_METRIC']])

                    # para o melhor embedding
                    df_best_embedding[key].append(embedding_mapping[df_arq_sample_bias.loc[idx, 'INPUT_EMBEDDING']])

                    # para o melhor optimizer
                    df_best_optimizer[key].append(optimizer_mapping[df_arq_sample_bias.loc[idx, 'OPTIMIZER']])

        # criar o caminho para salvar os arquivos
        file_path_ = f"{file_path}/input_data/experiment_{sample_id}/"
        if not os.path.exists(file_path_):
            os.makedirs(file_path_)

        # salvar os dicionários em arquivos csv
        df_best_accuracy = pd.DataFrame(df_best_accuracy)
        df_best_accuracy.to_csv(file_path_ + "best_accuracy_sample_" + str(sample_id) + ".csv", index=False)

        df_best_ansatz = pd.DataFrame(df_best_ansatz)
        df_best_ansatz.to_csv(file_path_ + "best_ansatz_sample_" + str(sample_id) + ".csv", index=False)

        df_unsupervised_metric = pd.DataFrame(df_unsupervised_metric)
        df_unsupervised_metric.to_csv(file_path_ + "unsupervised_metric_sample_" + str(sample_id) + ".csv", index=False)

        df_best_embedding = pd.DataFrame(df_best_embedding)
        df_best_embedding.to_csv(file_path_ + "best_embedding_sample_" + str(sample_id) + ".csv", index=False)

        df_best_optimizer = pd.DataFrame(df_best_optimizer)
        df_best_optimizer.to_csv(file_path_ + "best_optimizer_sample_" + str(sample_id) + ".csv", index=False)

    # salvando os mapeamentos em arquivos pickle
    with open(file_path + "input_data/ansatz_mapping.pkl", 'wb') as file:
        pickle.dump(ansatz_mapping, file)
    with open(file_path + "input_data/embedding_mapping.pkl", 'wb') as file:
        pickle.dump(embedding_mapping, file)
    with open(file_path + "input_data/unsupervised_mapping.pkl", 'wb') as file:
        pickle.dump(unsupervised_mapping, file)
    with open(file_path + "input_data/optimizer_mapping.pkl", 'wb') as file:
        pickle.dump(optimizer_mapping, file)


In [24]:
transformar_dataframe(df=df_sorted, file_path="./", target="TEST_ACCURACY")


Mapeamento de 'ARQUITETURA_ANSATZ' para números:
0_zero: 0
1_mochi: 1
2_yadi: 2
3_tatu: 3
4_wana: 4
5_tano: 5
6_samanu: 6
7_sambwadi: 7
8_nake: 8
9_divwe: 9
A: 10
B: 11
C: 12
D: 13
E: 14
F: 15
G: 16
H: 17
I: 18
J: 19
K: 20
L: 21
M: 22
N: 23
O: 24
alpha: 25
beta: 26
gamma: 27
hr: 28
phi: 29
ru: 30
ruu: 31
rx: 32
ry: 33
rz: 34
theta: 35

Mapeamento de 'INPUT_EMBEDDING' para números:
phasex: 0
phasey: 1
amplitude: 2

Mapeamento de 'UNSUPERVISED_METRIC' para números:
supervised: 0

Mapeamento de 'OPTIMIZER' para números:
SPSAOptimizer(10): 0
GradientDescentOptimizer(1): 1
NesterovMomentumOptimizer(0.1): 2
